# Align coordinates

If we're looking at epitopes in several related sequences, we may want to
align those sequences and analyse the epitopes in that alignment.
To help with this `epimap` provides `align_coordinate()`,
this function converts coordinates in an sequence without gaps to coordinates
in an version of the same sequence with gaps.

In [2]:

import pandas as pd
from epimap import map

# The aligned version of our sequence
aligned_seq = "ABCD--FGHIJKL--OP--STUVWXYZ"
# The unaligned version
seq = aligned_seq.replace("-","")

# A table of epitopes in sequence, positions are relative to the
# unaligned version
epitopes = pd.DataFrame({
        'start':[1,     4,      7,      10,     9,      11,     13],
        'end':  [4,     8,      11,     14,     13,     15,     17],
        'seq':  ["BCD", "FGHI", "IJKL", "LOPS", "KLOP", "OPST", "STUV"]
})


In [3]:

# epitope coords are correct
assert all(epitopes.apply(lambda x: seq[x.start:x.end]==x.seq, axis=1))


`align_coords()` takes an ungapped position, the sequence you would
like it aligned in, and a counting index. It returns the equivalent position in that aligned sequence.
Here position 4 in the unaligned sequence is equivalent to position 6
in the aligned sequence


In [5]:
i = 4
j = map.align_coords(i, aligned_seq, index=0)
print(f"{seq[i]} is position {i} in seq")
print(f"{aligned_seq[j]} is position {j} in aligned_seq")


F is position 4 in seq
F is position 6 in aligned_seq



A much faster way to use `align_coords()` is to `apply` it to several epitopes at once.


In [9]:
epitopes['newstart'] = epitopes.start.apply(map.align_coords, aligned_seq=aligned_seq, index=0)
epitopes['newend'] = epitopes.end.apply(map.align_coords, aligned_seq=aligned_seq, index=0)
print(aligned_seq)
epitopes

ABCD--FGHIJKL--OP--STUVWXYZ


,start,end,seq,newstart,newend
0,1,4,BCD,1,6
1,4,8,FGHI,6,10
2,7,11,IJKL,9,15
3,10,14,LOPS,12,20
4,9,13,KLOP,11,19
5,11,15,OPST,15,21
6,13,17,STUV,19,23


We can verify that the new positions are correct by slicing these
new coordinates from the `aligned_seq`. Note that python slicing
uses a zero index, and does not include the end index. Our example
positions follow this form, but if you are using a different index
and/or including the end index, you will have to account for this
in your slicing.M

In [11]:
epitopes['aligned_seq'] = epitopes.apply(lambda x: aligned_seq[x.newstart:x.newend], axis=1)
epitopes[['seq','aligned_seq']]


,seq,aligned_seq
0,BCD,BCD--
1,FGHI,FGHI
2,IJKL,IJKL--
3,LOPS,L--OP--S
4,KLOP,KL--OP--
5,OPST,OP--ST
6,STUV,STUV


## Unaligning coordinates
You may want to go in the opposite direction if you have the
epitope position in the aligned sequece, but want the position in the
unaligned sequence.

In [14]:
j = 9
i = map.unalign_coordinate(j, aligned_seq, index=0)

print(f"{aligned_seq[j]} is position {j} in aligned_seq")
print(f"{seq[i]} is position {i} in seq")

I is position 9 in aligned_seq
I is position 7 in seq


As before, we can `apply` this to several epitopes at once.
And verify that the unaligned positions match the original
positions we started with. And again, if you use these positions
to slice the sequences as we do here, be careful if you are not using
the python standard of zero index and not including the last index of
the slice.

In [18]:
epitopes['oldstart'] = epitopes.newstart.apply(map.unalign_coordinate, aligned_seq=aligned_seq, index=0)
epitopes['oldend'] = epitopes.newend.apply(map.unalign_coordinate, aligned_seq=aligned_seq, index=0)
epitopes[['start','oldstart','end','oldend']]

epitopes['unaligned_seq'] = epitopes.apply(lambda x: seq[x.oldstart:x.oldend], axis=1)
epitopes[['seq','unaligned_seq']]


,seq,unaligned_seq
0,BCD,BCD
1,FGHI,FGHI
2,IJKL,IJKL
3,LOPS,LOPS
4,KLOP,KLOP
5,OPST,OPST
6,STUV,STUV
